# Ingestão de Dados JSON da Landing Zone para a Bronze

 Este notebook realiza a leitura do arquivo JSON da landing zone, que contém um Bundle FHIR com uma lista de entradas no campo `entry`. 

 Em seguida, os recursos são extraídos, achatados e separados por tipo para serem salvos em tabelas Delta na camada Bronze.

 **Observações:**
 - Use o caminho adequado para o arquivo JSON da landing zone.
 - As tabelas são criadas com o modo "overwrite", garantindo que cada execução atualize os dados.


In [37]:
# Define a variável para o schema do Lakehouse Bronze
bronze_schema = "bronze_health"

# Diretório base onde os arquivos JSON estão salvos
base_directory = "Files/Raw/FHIRData"

StatementMeta(, 6835d3ec-3488-4691-888a-4372bcce0503, 39, Finished, Available, Finished)

In [38]:
display(mssparkutils.fs.ls("Files/Raw/FHIRData"))

StatementMeta(, 6835d3ec-3488-4691-888a-4372bcce0503, 40, Finished, Available, Finished)

[FileInfo(path=abfss://5aad9838-8ce9-4dff-af19-512a77e46917@onelake.dfs.fabric.microsoft.com/4ce8615c-a96e-4904-a59f-1c1fa48881a9/Files/Raw/FHIRData/2025, name=2025, size=0)]

In [39]:
from pyspark.sql.functions import col, explode
from pyspark.sql.types import StructType

# Função para listar arquivos recursivamente
def list_all_files(directory):
    file_list = []
    for item in mssparkutils.fs.ls(directory):
        if item.isDir:
            file_list.extend(list_all_files(item.path))
        else:
            file_list.append(item)
    return file_list


# Lista todos os arquivos a partir do diretório base
all_files = list_all_files(base_directory)

if not all_files:
    raise Exception(f"Nenhum arquivo encontrado em {base_directory}")

# Seleciona o arquivo mais recente com base no nome (assumindo que o nome possui a data/hora no formato AAAAMMDDHHMMSS)
latest_file = max(all_files, key=lambda x: x.name)
landing_zone_path = latest_file.path

print("Arquivo mais recente:", landing_zone_path)

# Leitura do JSON com a opção multiline para tratar arquivos com quebras de linha
df_raw = spark.read.option("multiline", "true").json(landing_zone_path)

# Visualiza o schema e algumas linhas para validação
#df_raw.printSchema()
#display(df_raw.limit(3))

StatementMeta(, 6835d3ec-3488-4691-888a-4372bcce0503, 41, Finished, Available, Finished)

Arquivo mais recente: abfss://5aad9838-8ce9-4dff-af19-512a77e46917@onelake.dfs.fabric.microsoft.com/4ce8615c-a96e-4904-a59f-1c1fa48881a9/Files/Raw/FHIRData/2025/02/23/20250223152342.json


 ## Extração das Entradas do Bundle

 O JSON lido é um Bundle FHIR que possui um campo `entry` contendo uma lista de registros. 

 Vamos explodir esse array para que cada entrada seja uma linha e extrair o campo `resource` de cada entrada.


In [40]:
# Explode o array "entry" para obter cada registro em uma linha
df_entries = df_raw.select(explode("entry").alias("entry"))

# Seleciona o objeto resource de cada entrada
df_resources = df_entries.select("entry.resource.*")

# Verifica o schema resultante e mostra alguns registros
#df_resources.printSchema()
display(df_resources.limit(5))


StatementMeta(, 6835d3ec-3488-4691-888a-4372bcce0503, 42, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4406665f-80f2-430a-aa74-d4cbe359676d)

## Função para Achatar o DataFrame

Muitas vezes os recursos possuem estruturas aninhadas (structs) que dificultam a análise tabular.

A função abaixo percorre as colunas do DataFrame e, para aquelas que são do tipo `struct`, extrai seus campos criando colunas com nomes concatenados.

In [41]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType

def flatten_df(nested_df):
    """
    Achata colunas do tipo struct para um DataFrame plano.
    """
    flat_cols = []
    for column in nested_df.columns:
        dtype = nested_df.schema[column].dataType
        if isinstance(dtype, StructType):
            for field in dtype.fields:
                flat_cols.append(col(f"{column}.{field.name}").alias(f"{column}_{field.name}"))
        else:
            flat_cols.append(col(column))
    return nested_df.select(flat_cols)

# Aplica o flatten para o DataFrame de recursos
df_flat = flatten_df(df_resources)

# Visualiza o schema e os primeiros registros já achatados
#df_flat.printSchema()
display(df_flat.limit(2))


StatementMeta(, 6835d3ec-3488-4691-888a-4372bcce0503, 43, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 200de59f-3a12-4550-90cc-47de0b8ec76b)

 ## Função para Salvar Cada Recurso como Tabela na Bronze

 A função `save_resource_table` filtra o DataFrame para um dado `resourceType`, aplica um flatten extra (caso haja novos aninhamentos) e salva o resultado como uma tabela Delta.

 O dicionário `resources_tables` define o mapeamento entre o tipo de recurso e o nome da tabela de destino.

In [ ]:
from pyspark.sql.functions import col, current_timestamp

def save_resource_table(df, resource_type, table_name):
    # Filtra os registros para o resourceType desejado
    df_filtered = df.filter(col("resourceType") == resource_type)
    
    # Se houver registros para esse recurso, aplica flatten, adiciona a data de atualização e salva a tabela
    if df_filtered.head(1):
        df_filtered_flat = flatten_df(df_filtered)
        # Adiciona a coluna 'update_date' com o timestamp atual
        df_filtered_flat = df_filtered_flat.withColumn("update_date", current_timestamp())
        df_filtered_flat.write.format("delta").mode("overwrite").saveAsTable(table_name)
        print(f"Tabela '{table_name}' criada com sucesso!")
    else:
        print(f"Nenhum registro encontrado para o resourceType: {resource_type}")

# Define a variável para o schema do Lakehouse Bronze
#bronze_schema = "bronze_health"

# Define os tipos de recursos e os respectivos nomes das tabelas na Bronze usando a variável
resources_tables = {
    "Patient": f"{bronze_schema}.Patients",
    "Encounter": f"{bronze_schema}.Encounters",
    "Condition": f"{bronze_schema}.Conditions",
    "DiagnosticReport": f"{bronze_schema}.DiagnosticReports",
    "DocumentReference": f"{bronze_schema}.DocumentReferences",
    "Claim": f"{bronze_schema}.Claims",
    "ExplanationOfBenefit": f"{bronze_schema}.Explanations",
    "MedicationRequest": f"{bronze_schema}.MedicationRequests",
    "Device": f"{bronze_schema}.Devices",
    "SupplyDelivery": f"{bronze_schema}.SupplyDeliveries",
    "CareTeam": f"{bronze_schema}.CareTeams",
    "CarePlan": f"{bronze_schema}.CarePlans",
    "Observation": f"{bronze_schema}.Observations",
    "Procedure": f"{bronze_schema}.Procedures",
    "Medication": f"{bronze_schema}.Medications",
    "MedicationAdministration": f"{bronze_schema}.MedicationAdministrations",
    "Immunization": f"{bronze_schema}.Immunizations",
    "ImagingStudy": f"{bronze_schema}.ImagingStudies",
    "Provenance": f"{bronze_schema}.Provenances"
}

# Itera sobre os tipos de recurso e salva cada tabela com a coluna de atualização
for resource_type, table_name in resources_tables.items():
    save_resource_table(df_flat, resource_type, table_name)


StatementMeta(, 6835d3ec-3488-4691-888a-4372bcce0503, 44, Finished, Available, Finished)

Tabela 'bronze_health.Patients' criada com sucesso!
Tabela 'bronze_health.Encounters' criada com sucesso!
Tabela 'bronze_health.Conditions' criada com sucesso!
Tabela 'bronze_health.DiagnosticReports' criada com sucesso!
Tabela 'bronze_health.DocumentReferences' criada com sucesso!
Tabela 'bronze_health.Claims' criada com sucesso!
Tabela 'bronze_health.Explanations' criada com sucesso!
Tabela 'bronze_health.MedicationRequests' criada com sucesso!
Tabela 'bronze_health.Devices' criada com sucesso!
Tabela 'bronze_health.SupplyDeliveries' criada com sucesso!
Tabela 'bronze_health.CareTeams' criada com sucesso!
Tabela 'bronze_health.CarePlans' criada com sucesso!
Tabela 'bronze_health.Observations' criada com sucesso!
Tabela 'bronze_health.Procedures' criada com sucesso!
Tabela 'bronze_health.Medications' criada com sucesso!
Tabela 'bronze_health.MedicationAdministrations' criada com sucesso!
Tabela 'bronze_health.Immunizations' criada com sucesso!
Tabela 'bronze_health.ImagingStudies' cri

## Verificação

 Você pode consultar cada tabela criada para confirmar os dados.


 Exemplo: Visualizando os primeiros registros da tabela de pacientes.


In [43]:
display(spark.table("bronze_health.patients").limit(2))


StatementMeta(, 6835d3ec-3488-4691-888a-4372bcce0503, 45, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 910569fd-3175-4410-855c-c5f09a311457)